## 1. Pydantic Basics: Creating and using models

Pydantic models are the foundation of data validation in python. They use python type annotations to define the structure and validate data at anytime. 
Here's a detailed exploration of basic model creation with several examples.

In [2]:
# Dataclass is a decorator that is used to define some attributes of a class. Dataclass is decorator that automatically generates special methods 
# like __init__(), __repr__(), __eq__(), and others based on the class attributes.
# Dataclass helps to define classes that are primarily used to store data, but no validation or business logic.

from dataclasses import dataclass

@dataclass
class Person():
    name: str
    age: int
    city: str

In [3]:
person = Person(name="kiran", age=35, city="bangalore")
print(person)
person1 = Person(name="kiran", age=35, city=40)
print(person1)

Person(name='kiran', age=35, city='bangalore')
Person(name='kiran', age=35, city=40)


Pydantic is a data validation and settings management library for Python, which uses Python type annotations.
When you are inheriting the base model then it is Data Model

In [5]:
from pydantic import BaseModel

class PersonModel(BaseModel):
    name: str
    age: int
    city: str

In [6]:
person = PersonModel(name="kiran", age=35, city="bangalore")
print(person)

name='kiran' age=35 city='bangalore'


In [8]:
try:
    person1 = PersonModel(name="kiran", age=35, city=40)
    print(person1)
except ValueError as e:
    print(f"Validation error: {e}")

Validation error: 1 validation error for PersonModel
city
  Input should be a valid string [type=string_type, input_value=40, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type


## 2. Models with Optional Fields
Add optional fields using python's pydantic type.
Pydantic validates types even for optional fields when values are provided

In [9]:
from typing import List, Optional

class Employee(BaseModel):
    name: str
    age: int
    city: str
    skills: List[str] = []
    manager: Optional[str] = None  # Optional field, can be None as default value
    is_active: bool = True  # Default value is True
    salary: float = 0.0  # Default value is 0.0

In [10]:
# type casting is happening automatically for salary
emp1 = Employee(name="kiran", age=35, city="bangalore", skills=["python", "django"], salary = 1000)
print(emp1)

emp2 = Employee(name="kiran", age=35, city="bangalore", skills=["python", "django"], salary = "1000")
print(emp2)

name='kiran' age=35 city='bangalore' skills=['python', 'django'] manager=None is_active=True salary=1000.0
name='kiran' age=35 city='bangalore' skills=['python', 'django'] manager=None is_active=True salary=1000.0


## 3. Model with Nested models
Create complex structures with nested models.

In [12]:
from pydantic import BaseModel

class Address(BaseModel):
    street: str
    city: str
    state: str
    zip_code: str

class EmployeeWithAddress(BaseModel):
    name: str
    age: int
    address: Address  # Nested model for address

In [13]:
emp1 = EmployeeWithAddress(name="kiran", age=35, address={"street": "123 Main St", "city": "Bangalore", "state": "KA", "zip_code": "560001"})
print(emp1)

name='kiran' age=35 address=Address(street='123 Main St', city='Bangalore', state='KA', zip_code='560001')


## 4. Pydantic Fields: Customerization and constraints

The Field function in pydantic enhances model with fields beyond basic type hints by allowing you to specify validation rules, default values, aliases and more. 

In [15]:
from pydantic import BaseModel, Field

class Customer(BaseModel):
    name: str = Field(default="kiran", max_length=50, description="The name of the customer")
    age: int = Field(default=24, ge=0,lt=100, description="The age of the customer, must be a non-negative integer")
    email: str = Field(default="kiran@gmail.com", pattern=r'^[\w\.-]+@[\w\.-]+\.\w+$', description="The email address of the customer")

In [16]:
try:
    # Validation Error: Input should be less than 100 [type=less_than, input_value=101, input_type=int]
    # cust1 = Customer(name="k1", age=101, email="kiran.k@gmail.com")
    cust2 = Customer(name="k1", age=10, email="kiran.k@gmail.com")
    print(cust2)

except ValueError as e:
    print(f"Validation error: {e}")

name='k1' age=10 email='kiran.k@gmail.com'


In [18]:
# model to read environment variables for username and password
from pydantic_settings import BaseSettings

class User(BaseSettings):
    username: str
    password: str

    class Config:
        # This will allow the model to read environment variables for username and password
        env_prefix = 'USER_'
        env_file = '.env'  # Optional, if you want to load from a .env file

# Example usage 
user = User(username="admin", password="secret")
print(user)

# You can also load the model from environment variables
import os
from dotenv import load_dotenv
# Ensure you have a .env file with USER_USERNAME and USER_PASSWORD variables
load_dotenv()
user_from_env = User()
print(user_from_env)

username='admin' password='secret'
username='admin' password='secret'


In [ ]:
# n Pydantic, the ... inside a Field indicates that the field is required. 
# It is a shorthand for specifying that the field must be provided when creating an instance of the model. 
# If the field is not provided, Pydantic will raise a validation error.

from pydantic import model_validator

class Customer(BaseModel):
    name: str=Field(..., description="The name of the customer, required field")
    age: int=Field(..., ge=0, lt=100, description="The age of the customer, must be a non-negative integer")
    email: str=Field(default="", description="User email address")
    
    @model_validator(mode='after')
    def set_email(self):
        if not self.email:
            self.email = f"{self.name}@example.com"
        return self

# Example usage
try:
    cust = Customer(name="Kiran", age=30)
    print(cust)
except ValueError as e:
    print(f"Validation error: {e}")

name='Kiran' age=30 email='name@example.com'


In [ ]:
# Using default_factory with lambda: Examples where lambda works correctly
# default_factory is perfect for generating dynamic defaults that don't depend on other fields
# The lambda function is called each time a new instance is created

from datetime import datetime
import uuid
import random

class Product(BaseModel):
    name: str = Field(..., description="Product name")
    price: float = Field(..., ge=0, description="Product price")
    
    # Example 1: Generate a unique ID using lambda
    product_id: str = Field(
        default_factory=lambda: str(uuid.uuid4()),
        description="Unique product identifier"
    )
    
    # Example 2: Generate current timestamp using lambda
    created_at: datetime = Field(
        default_factory=lambda: datetime.now(),
        description="Product creation timestamp"
    )
    
    # Example 3: Generate a random discount percentage
    discount_percent: float = Field(
        default_factory=lambda: round(random.uniform(0, 20), 2),
        description="Random discount percentage"
    )
    
    # Example 4: Initialize an empty list (important: use lambda to avoid shared mutable defaults)
    tags: list[str] = Field(
        default_factory=list,  # Can use list directly or lambda: []
        description="Product tags"
    )

# Create multiple products - each gets unique values
print("Product 1:")
prod1 = Product(name="Laptop", price=999.99)
print(prod1)
print(f"Product ID: {prod1.product_id}")
print(f"Created at: {prod1.created_at}")
print(f"Discount: {prod1.discount_percent}%")
print()

print("Product 2:")
prod2 = Product(name="Mouse", price=29.99)
print(prod2)
print(f"Product ID: {prod2.product_id}")
print(f"Created at: {prod2.created_at}")
print(f"Discount: {prod2.discount_percent}%")
print()

# Note: Each instance gets different values because lambda is called for each instance
print(f"Different IDs: {prod1.product_id != prod2.product_id}")
print(f"Different timestamps: {prod1.created_at != prod2.created_at}")
print(f"Different discounts: {prod1.discount_percent != prod2.discount_percent}")

In [4]:
# Default Factory inside field: Default_factory is correctly used to provide a default email value dynamically. 
# If you want to customize the default further (e.g., based on the name field), you can modify the lambda function accordingly.
from pydantic import  Field, BaseModel
class User(BaseModel):
    email: str = Field(default="kiran@gmail.com", pattern=r'^[\w\.-]+@[\w\.-]+\.\w+$', description="The email address of the customer")
    username: str = Field(default_factory=lambda data: f"{data['email'].split('@')[0]}")

user = User(email="random@gmail.com")
print(user.username)
user = User()
print(user.username)


random
kiran


In [26]:
# By default Pydantic won't validate default values. The Validate_default field parameter can be used to enable this behaviour.
from pydantic import BaseModel, Field, ValidationError
class User(BaseModel):
    age: int = Field(default="Twelve", validate_default=True)
try:
    user = User()
except ValidationError as e:
    print(e)

1 validation error for User
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='Twelve', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/int_parsing


In [27]:
Customer.model_json_schema()

{'properties': {'name': {'description': 'The name of the customer, required field',
   'title': 'Name',
   'type': 'string'},
  'age': {'description': 'The age of the customer, must be a non-negative integer',
   'exclusiveMaximum': 100,
   'minimum': 0,
   'title': 'Age',
   'type': 'integer'},
  'email': {'description': 'User email address',
   'title': 'Email',
   'type': 'string'}},
 'required': ['name', 'age'],
 'title': 'Customer',
 'type': 'object'}

In [28]:
Customer.model_json_schema(mode='validation')

{'properties': {'name': {'description': 'The name of the customer, required field',
   'title': 'Name',
   'type': 'string'},
  'age': {'description': 'The age of the customer, must be a non-negative integer',
   'exclusiveMaximum': 100,
   'minimum': 0,
   'title': 'Age',
   'type': 'integer'},
  'email': {'description': 'User email address',
   'title': 'Email',
   'type': 'string'}},
 'required': ['name', 'age'],
 'title': 'Customer',
 'type': 'object'}